# Testing regression model

In [1]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from models.utils import identify_device
from models.regression_utils import test_regression_model, load_regression_dataset, print_metrics_table
from models.Regression_models import CNN1D_Regressor

In [2]:
import os 
os.getcwd()

'c:\\Users\\VECSEL\\Documents\\Theo\\ML\\THz-TD-CNN'

In [3]:
device = torch.device('cpu')
model = CNN1D_Regressor()
trained_model_name = 'regression_model_train_on_100k_clean_lr0_0001_epochs_200'
unseen_dataset_name = 'unseen_3_layer_nonoise_n1to8'
training_dataset = 'train_3_layer_nonoise_n1to8'

dataset = load_regression_dataset(f'regression_data/{unseen_dataset_name}.pt')
unseen_loader = DataLoader(dataset, batch_size=256)

model.load_state_dict(torch.load(f'trained_models/{trained_model_name}.pth', map_location=torch.device('cpu')))
model.to(device)

metrics, results = test_regression_model(model, unseen_loader, device)
print_metrics_table(metrics, num_layers=3)

Loading data from regression_data/unseen_3_layer_nonoise_n1to8.pt
Dataset loaded successfully!
Number of samples: 5000
Shape of input pulse: torch.Size([1, 1024]), target vector: torch.Size([9])

=== Scaled-domain metrics ===
| Param   |   RMSE |    MAE |     R² |
|---------|--------|--------|--------|
| n1      | 0.2105 | 0.1738 | 0.459  |
| k1      | 0.2352 | 0.1951 | 0.3325 |
| d1      | 0.2282 | 0.1892 | 0.3735 |
| n2      | 0.1098 | 0.0816 | 0.8493 |
| k2      | 0.2039 | 0.1626 | 0.5049 |
| d2      | 0.1748 | 0.1358 | 0.6215 |
| n3      | 0.21   | 0.1723 | 0.4604 |
| k3      | 0.2357 | 0.1941 | 0.3284 |
| d3      | 0.2303 | 0.1898 | 0.3635 |
| Mean    | 0.2043 | 0.1661 | 0.477  |

=== Unscaled-domain metrics ===
| Param   |   RMSE |    MAE |     R² |     NRMSE |   MaxErr |   P95Err |
|---------|--------|--------|--------|-----------|----------|----------|
| n1      | 0.8521 | 0.4149 | 0.8776 |    0.1217 |   3.9773 |   2.1876 |
| k1      | 0.9453 | 0.4619 | 0.8488 |    9.3597 |   4

In [9]:
len(results['preds_unscaled'])

9